<a href="https://colab.research.google.com/github/monsterhunters/Stable-Diffusion/blob/main/New_SD_V1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

V1.8

In [ ]:
#@title # **Install** 🚀
# All necessary imports goes here
from IPython.display import clear_output, display, HTML
import os, subprocess, time, glob
import zipfile
import shutil
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote
from google.colab.output import eval_js
%cd /content

import time
try:
  start_colab
except:
  start_colab = int(time.time())-5

print("\033[96m") #Cyan text
# Check if gpu exist, stop if don't.
try:
  output
except:
  print('⌚ Checking GPU...', end='')
  output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
  if "name" in output:
    gpu_name = output[5:]
    print('\r✅ Current GPU:', gpu_name, flush=True)
  else:
    print('\r\033[91m❎ ERROR: No GPU detected. Please do step below to enable.\n', flush=True)
    display(HTML("<img src='https://i.ibb.co/HC9KH17/NVIDIA-Share-23-01-02-173037.png' width='800px'/>"))
    print('\033[91m\nIf it says "Cannot connect to GPU backend", meaning you\'ve either reached free usage limit. OR there\'s no gpu available.\n\nDon\'t mind me... I\'m destroying your current session for your own good...')
    display(HTML("<img src='https://media.tenor.com/E9omRGF7x0AAAAAC/hitori-gotou-bocchi-rock.gif' width='500px'/>"))
    time.sleep(5)
    from google.colab import runtime
    runtime.unassign()



codemaster_addon = True #@param{type: "boolean"}
xformers = True #@param{type: "boolean"}

from google.colab import drive
drive.mount('/content/drive')

start_install = int(time.time())

def my_function():
    # Clone the repository
    os.system('git clone https://github.com/monsterhunters/stablecode.git')

    # Download and install dependencies
    os.system('wget https://huggingface.co/NoCrypt/fast-repo/resolve/main/ubuntu_deps.zip ; unzip ubuntu_deps.zip -d ./deps ; dpkg -i ./deps/* ; rm -rf ubuntu_deps.zip /content/deps/')

    # Download files using aria2c
    aria_command = (
        "aria2c -i- -j5 -x16 -s16 -k1M -c <<EOF\n"
        "https://huggingface.co/NoCrypt/fast-repo/resolve/main/dep.tar.lz4\n"
        " out=dep.tar.lz4\n"
        "https://huggingface.co/bigbossmonster/ext/resolve/main/ext4.tar.lz4\n"
        " out=ext4.tar.lz4\n"
        "https://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4\n"
        " out=cache.tar.lz4\n"
        "https://huggingface.co/bigbossmonster/ext/resolve/main/ext3.tar.lz4\n"
        " out=ext3.tar.lz4\n"
        "EOF"
    )
    os.system(aria_command)
    print('Finishing the setup.. ')
    # Extract files using tar and lz4
    os.system('tar -xI lz4 -f dep.tar.lz4 --overwrite-dir --directory=/usr/local/lib/python3.10/dist-packages/')
    os.system('tar -xI lz4 -f ext3.tar.lz4 --overwrite-dir --directory=/content/stablecode/')
    os.system('tar -xI lz4 -f ext4.tar.lz4 --overwrite-dir --directory=/content/stablecode/')
    os.system('tar -xI lz4 -f cache.tar.lz4 --directory=/')

    !wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
    %env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

    os.environ["SAFETENSORS_FAST_GPU"]='1'
    os.environ["CUDA_MODULE_LOADING"]="LAZY"
    os.environ["colab_url"] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["PYTHONWARNINGS"] = "ignore"
    # %env PYTHONDONTWRITEBYTECODE=1
    os.environ["TCMALLOC_AGGRESSIVE_DECOMMIT"] = "t"
    # os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "garbage_collection_threshold:0.9,max_split_size_mb:512"
    # !apt -y update -qq

    # Remove downloaded archive files
    os.system('rm -rf /content/dep.tar.lz4 /content/ext3.tar.lz4 /content/ext4.tar.lz4 /content/cache.tar.lz4')
    print('Successfully install')



# Call the function
print('Unpacking.. Please be patient')
my_function()

#!echo -n {start_colab} > stablecode/static/colabTimer.txt
#if not os.path.exists("/content/stablecode"):

install_time = timedelta(seconds=time.time()-start_install)
print("\r🚀 Finished unpacking. Took","%02d:%02d:%02d ⚡\n" % (install_time.seconds / 3600, (install_time.seconds / 60) % 60, install_time.seconds % 60), end='', flush=True)
# Colab 🤝 Gradio (Colab timer integration for gradio)
!echo -n {start_colab} > /content/stablecode/static/colabTimer.txt
print("🤝 Colab timer integration complete! You can see your colab time inside webui.")



if codemaster_addon:


  # Function to copy files to the destination directory, checking if they exist already
  def copy_files(files, source_directory, destination_directory):
      for file in files:
          source_file = os.path.join(source_directory, file)
          destination_file = os.path.join(destination_directory, file)
          os.makedirs(destination_directory, exist_ok=True)  # Create the destination directory if it doesn't exist
          shutil.copy2(source_file, destination_file)

  def extract_all_zips(wildcards_directory, destination_wildcards_directory):
      for file in os.listdir(wildcards_directory):
          if file.endswith('.zip'):
              file_path = os.path.join(wildcards_directory, file)
              with zipfile.ZipFile(file_path, 'r') as zip_ref:
                  zip_ref.extractall(destination_wildcards_directory)

  # Update the paths and add the necessary steps

  folder_path = '/content/stablecode/extensions/Umi-AI-debloat'

  if os.path.exists(folder_path):
      shutil.rmtree(folder_path)


  if not os.path.exists("/content/stablecode/extensions/sd-dynamic-prompts"):
      %cd /content/stablecode/extensions
      os.system('git clone https://github.com/adieyal/sd-dynamic-prompts.git')

  if not os.path.exists("/content/stablecode/plugin-for-SD"):
      %cd /content/stablecode
      os.system('git clone https://github.com/monsterhunters/plugin-for-SD.git')



  scripts_directory = "/content/stablecode/plugin-for-SD/scripts"  # Source directory for Python files
  wildcards_directory = "/content/stablecode/plugin-for-SD/wildcards"  # Source directory for text files
 # modules_directory = "/content/stablecode/plugin-for-SD/modules"  # Source directory for text files
  embeddings_directory = "/content/stablecode/plugin-for-SD/embeddings"  # Source directory for text files
  destination_scripts_directory = "/content/stablecode/scripts"  # Destination directory for Python files
  destination_wildcards_directory = "/content/stablecode/extensions/sd-dynamic-prompts/wildcards"  # Destination directory for text files
  destination_modules_directory = "/content/stablecode/modules"  # Destination directory for text files
  destination_embeddings_directory = "/content/stablecode/embeddings"  # Destination directory for text files

  # Rest of the code...

  # Extract all ZIP files from wildcards_directory
  extract_all_zips(wildcards_directory, destination_wildcards_directory)


  def copy_files(files, source_directory, destination_directory):
      for file in files:
          source_file = os.path.join(source_directory, file)
          destination_file = os.path.join(destination_directory, file)
          os.makedirs(destination_directory, exist_ok=True)  # Create the destination directory if it doesn't exist
          shutil.copy2(source_file, destination_file)
  # Get a list of all text files and Python files in the scripts directory
  script_files = [file for file in os.listdir(scripts_directory) if file.endswith(".py")]
  # Copy Python files to the destination scripts directory, checking if they exist already
  copy_files(script_files, scripts_directory, destination_scripts_directory)
  # Get a list of all text files in the wildcards directory
  wildcard_files = [file for file in os.listdir(wildcards_directory) if file.endswith(".txt")]
  # Copy text files to the destination wildcards directory, checking if they exist already
  copy_files(wildcard_files, wildcards_directory, destination_wildcards_directory)
  # Get a list of all Python files in the modules directory
#  module_files = [file for file in os.listdir(modules_directory) if file.endswith(".py")]
  # Copy Python files to the destination modules directory, checking if they exist already
#  copy_files(module_files, modules_directory, destination_modules_directory)

  embeddings_files = [file for file in os.listdir(embeddings_directory) if file.endswith((".pt", ".bin"))]
  # Copy Python files to the destination modules directory, checking if they exist already
  copy_files(embeddings_files, embeddings_directory, destination_embeddings_directory)

#!rm rf "/content/stablecode/extensions/sd-photopea-embed-lazy"

if xformers:
  !pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
  !pip install -q xformers==0.0.20 triton==2.0.0 -U
  !pip install pyre-extensions==0.0.29

#Fix_colab
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!apt-get install -f
#!cloudflared --version
!mv /usr/local/bin/cloudflared /usr/local/bin/wex


shutil.rmtree("/content/stablecode/extensions/sd-photopea-embed-lazy")

In [ ]:
#@title # **RUN** 🚀
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nUI finished loading, trying to launch (if it gets stuck here  is having issues)\n")

  p = subprocess.Popen(["wex", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access UI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

!python launch.py --enable-insecure-extension-access --ckpt-dir '/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/checkpoints' --vae-dir '/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/vae' --embeddings-dir '/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/embeddings' --lora-dir '/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/loras' --esrgan-models-path '/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/CodeUI/models/upscale_models'  --controlnet-dir '/content/drive/Shareddrives/BSSG-RCN-(70)/HTZ/controlnet'